In [1]:
import pandas as pd
import numpy as np
from statsmodels.stats.proportion import proportion_confint, samplesize_confint_proportion
import matplotlib.pyplot as plt
import scipy as sp

# Доверительные интервалы. Главный тест первой недели
## Правило трех сигм
Давайте уточним правило трёх сигм. Утверждение: 99.7% вероятностной массы случайной величины $X∼N(μ,σ2)$ лежит в интервале $\mu\pm c \cdot \sigma$. Чему равно точное значение константы $c$? Округлите ответ до четырёх знаков после десятичной точки.

In [2]:
alpha = 0.003
c = np.round(sp.stats.norm.ppf(1 - alpha / 2) , 4)
print (c)

2.9677


## Приём аспирина спасает жизни
В пятилетнем рандомизированном исследовании Гарвардской медицинской школы 11037 испытуемых через день принимали аспирин, а ещё 11034 — плацебо. Исследование было слепым, то есть, испытуемые не знали, что именно они принимают.

За 5 лет инфаркт случился у 104 испытуемых, принимавших аспирин, и у 189 принимавших плацебо.

Оцените, насколько вероятность инфаркта снижается при приёме аспирина. Округлите ответ до четырёх знаков после десятичной точки.

In [3]:
n_asp = 11037
dead_asp = 104
n_plac = 11034
dead_plac = 189
alpha = 0.05

In [4]:
result = np.round(dead_plac / n_plac - dead_asp / n_asp, 4)  
print('Настолько уменьшится вероятность инфаркта при приеме аспирина', result)

Настолько уменьшится вероятность инфаркта при приеме аспирина 0.0077


##  Доверительный интервал
Постройте теперь 95% доверительный интервал для снижения вероятности инфаркта при приёме аспирина. Чему равна его верхняя граница? Округлите ответ до четырёх знаков после десятичной точки.

In [5]:
aspirin = np.full(n_asp, 0)
aspirin[np.arange(dead_asp)] = 1
placebo = np.full(n_plac, 0)
placebo[np.arange(dead_plac)] = 1

In [6]:
def proportions_confint_diff_ind(sample1, sample2, alpha = 0.05):    
    z = sp.stats.norm.ppf(1 - alpha / 2.)   
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    left_boundary = np.round(left_boundary, 4)
    right_boundary = np.round(right_boundary, 4)
    return (left_boundary, right_boundary)

In [7]:
print("confidence interval: [%f, %f]" % proportions_confint_diff_ind(placebo, aspirin))

confidence interval: [0.004700, 0.010700]


## Шансы для бернуллиевских величин
Продолжим анализировать данные эксперимента Гарвардской медицинской школы.

Для бернуллиевских случайных величин $X∼Ber(p)$ часто вычисляют величину $\frac{p}{1-p} $, которая называется шансами (odds). Чтобы оценить шансы по выборке, вместо $p$ нужно подставить $\hat{p}$. Например, шансы инфаркта в контрольной группе, принимавшей плацебо, можно оценить как

$$\frac{\frac{189}{11034}}{1-\frac{189}{11034}} = \frac{189}{11034-189}\approx 0.0174 $$

Оцените, во сколько раз понижаются шансы инфаркта при регулярном приёме аспирина. Округлите ответ до четырёх знаков после десятичной точки.

In [8]:
odds_asp = dead_asp / (n_asp - dead_asp)
odds_plac = dead_plac / (n_plac - dead_plac)
odds_ratio = np.round(odds_plac / odds_asp, 4)
print('Настолько уменьшится вероятность инфаркта при приеме аспирина (шансы)', odds_ratio)

Настолько уменьшится вероятность инфаркта при приеме аспирина (шансы) 1.8321


## Бутстрап
Величина, которую вы оценили в предыдущем вопросе, называется отношением шансов. Постройте для отношения шансов 95% доверительный интервал с помощью бутстрепа. Чему равна его нижняя граница? Округлите ответ до 4 знаков после десятичной точки.

Чтобы получить в точности такой же доверительный интервал, как у нас:

- составьте векторы исходов в контрольной и тестовой выборках так, чтобы в начале шли все единицы, а потом все нули;
- установите random seed=0;
- сделайте по 1000 псевдовыборок из каждой группы пациентов с помощью функции get_bootstrap_samples.

In [9]:
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples
def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    boundaries = np.round(boundaries, 4)
    return boundaries
def get_odds(data):
    summ = data.sum() 
    n = len(data)  
    odds_ratio = summ / (n - summ)
    return odds_ratio

In [10]:
b_count = 1000
np.random.seed(0)

placebo_odds = np.array(list(map(get_odds, get_bootstrap_samples(placebo, b_count))))
aspirin_odds = np.array(list(map(get_odds, get_bootstrap_samples(aspirin, b_count))))

print("95% confidence interval for placebo_odds",  stat_intervals(placebo_odds, 0.05))
print("95% confidence interval for aspirin_odds",  stat_intervals(aspirin_odds, 0.05))
print("95% confidence interval for placebo_odds - aspirin_odds",  stat_intervals(placebo_odds / aspirin_odds, 0.05))


95% confidence interval for placebo_odds [0.0148 0.0199]
95% confidence interval for aspirin_odds [0.0078 0.0113]
95% confidence interval for placebo_odds - aspirin_odds [1.4629 2.3509]


## Результат
Был закреплен материал из лекций. И заодно стал более внимательно относиться к округлениям)